In [1]:
import numpy as np
import pandas as pd
import math

import torch
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from pytorch_lightning import seed_everything
seed_everything(42, workers=True)
torch.use_deterministic_algorithms(True, warn_only=True)


import dcor

from datetime import datetime

from torch import nn

import os 



[rank: 0] Global seed set to 42


In [2]:
# Create device agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

In [3]:
n=2000
for model1 in range(1,4):
    for model2 in range(1,4):
        dcor_list=[]
        for t in range(1,101):
            x_train=pd.read_csv("./data/model" + str(model1) + "-" + str(model2) + "-" + str(n) + "/x_train_" + str(t) + ".csv")
            x_train=x_train.drop('Unnamed: 0', axis=1)
            y_train=pd.read_csv("./data/model" + str(model1) + "-" + str(model2) + "-" + str(n) + "/y_train_" + str(t) + ".csv")
            y_train=y_train.drop('Unnamed: 0', axis=1)
            x_test=pd.read_csv("./data/model" + str(model1) + "-" + str(model2) + "-" + str(n) + "/x_test_" + str(t) + ".csv")
            x_test=x_test.drop('Unnamed: 0', axis=1)
            y_test=pd.read_csv("./data/model" + str(model1) + "-" + str(model2) + "-" + str(n) + "/y_test_" + str(t) + ".csv")
            y_test=y_test.drop('Unnamed: 0', axis=1)
            z_test=pd.read_csv("./data/model" + str(model1) + "-" + str(model2) + "-" + str(n) + "/z_test_" + str(t) + ".csv")
            z_test=z_test.drop('Unnamed: 0', axis=1)


            n=x_train.shape[0]
            p=x_train.shape[1]
            res_d=1
            m=2
            x_train = torch.tensor(x_train.values).to(torch.float)
            y_train = torch.tensor(y_train.values).to(torch.float)
            x_test = torch.tensor(x_test.values).to(torch.float)
            y_test = torch.tensor(y_test.values).to(torch.float)
            mse_loss = nn.MSELoss()
            
            class nn_dr_reg_model(nn.Module):
                def __init__(self, input_features, output_features, dim_red_features, hidden_units_d, hidden_units_e, dim_red_layers, ens_reg_layers):
                    super().__init__()
                    model_dim_red=[]
                    model_dim_red.append(nn.Linear(in_features=input_features, 
                                                out_features=hidden_units_d))
                    model_dim_red.append(nn.ReLU())
                    for i in range(1,dim_red_layers):
                        model_dim_red.append(nn.Linear(in_features=hidden_units_d, 
                                                    out_features=hidden_units_d))
                        model_dim_red.append(nn.ReLU())
                    model_dim_red.append(nn.Linear(in_features=hidden_units_d, 
                                                out_features=dim_red_features))
                    self.dim_red_layer_stack = nn.Sequential(*model_dim_red)

                    model_ens_reg=[]
                    model_ens_reg.append(nn.Linear(in_features=dim_red_features, out_features=hidden_units_e))
                    model_ens_reg.append(nn.ReLU())
                    for i in range(1,ens_reg_layers):
                        model_ens_reg.append(nn.Linear(in_features=hidden_units_e, out_features=hidden_units_e))
                        model_ens_reg.append(nn.ReLU())
                    model_ens_reg.append(nn.Linear(in_features=hidden_units_e, out_features=output_features))
                    self.ens_reg_layer_stack = nn.Sequential(*model_ens_reg)
                
                def forward(self, x):
                    suff_predictor = self.dim_red_layer_stack(x)
                    ens_output = self.ens_reg_layer_stack(suff_predictor)
                    return ens_output, suff_predictor
                    

            model_nn = nn_dr_reg_model(input_features=p, 
                                    output_features=1, 
                                    dim_red_features=res_d, 
                                    hidden_units_d=300,
                                    hidden_units_e=300,
                                    dim_red_layers=4, 
                                    ens_reg_layers=4
                                    ).to(device)
            model_nn
            optimizer = torch.optim.Adam(model_nn.parameters(), 
                                        lr=0.001)
            epochs = 200
            x_train, y_train = x_train.to(device), y_train.to(device)
            x_test, y_test = x_test.to(device), y_test.to(device)
            for epoch in range(epochs):
                ### Training
                model_nn.train()
                y_pred_train, y_suff_train = model_nn(x_train) 
                loss = mse_loss(y_pred_train, y_train) 
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

                ### Testing
                model_nn.eval()
                y_pred_test, y_suff_test = model_nn(x_test)
                loss_test = mse_loss(y_pred_test, y_test) 
                #dcor_test = dcor.distance_correlation(y_suff_test.detach().numpy(),z_test)

                if epoch % 25 == 0:
                    #print(f"Epoch: {epoch} | Loss: {loss:.5f} | Test Loss: {loss_test:.5f} | Dcor: {dcor_test:.5f}")
                    now = datetime.now()
                    current_time = now.strftime("%H:%M:%S")
                    #print("Current Time =", current_time)
            model_nn.eval()
            with torch.inference_mode():
                y_pred_test, y_suff_test = model_nn(x_test)
            y_suff_test=y_suff_test.numpy()
            dcor_current=dcor.distance_correlation(np.float64(y_suff_test),np.float64(z_test.to_numpy()))
            dcor_list.append(dcor_current)
            print(model1, model2, t, dcor_current)
        dcor_list_df=pd.DataFrame(dcor_list)
        dcor_list_df.to_csv("./results-BENN/result-" + str(model1) + "-" + str(model2) + "-" + str(n) + ".csv")
        print(model1, model2, np.mean(dcor_list), np.std(dcor_list))


1 1 1 0.9370502796106446
1 1 2 0.9393286721792483
1 1 3 0.9341674010060695
1 1 4 0.9241937165845101
1 1 5 0.9379997811378074
1 1 6 0.9410960068007259
1 1 7 0.932917382309664
1 1 8 0.9298286487659617
1 1 9 0.9409701329285639
1 1 10 0.9449925544140583
1 1 11 0.9438152436439751
1 1 12 0.9371018414761393
1 1 13 0.9408907248294786
1 1 14 0.949552843173084
1 1 15 0.9356455977358272
1 1 16 0.9391286579792901
1 1 17 0.933830011045357
1 1 18 0.9333071686623762
1 1 19 0.9359198241500771
1 1 20 0.9347323434361753
1 1 21 0.9465249398649703
1 1 22 0.9442122778268046
1 1 23 0.9354467427860673
1 1 24 0.9344272641049706
1 1 25 0.9382976962297943
1 1 26 0.9216799230757045
1 1 27 0.9315796697573493
1 1 28 0.9354327493319733
1 1 29 0.9326196322148275
1 1 30 0.9540578681653719
1 1 31 0.9331608812346783
1 1 32 0.9446004829714543
1 1 33 0.9424885785701645
1 1 34 0.9332870447092849
1 1 35 0.937452177009404
1 1 36 0.9416315052949515
1 1 37 0.9363239515092139
1 1 38 0.9417920178229494
1 1 39 0.9340305999990423